<h2>About this Exercise</h2>
<p>In the preceding activity, you derived a Euclidean distance matrix. Now that you have calculated the distance between points in terms of matrix operations, you are ready to write an efficient program that leverages NumPy's optimized functions. In this code exercise, rather than using loops, you will write a function to compute Euclidean distances between sets of vectors using NumPy functions.</p>

<h3>Evaluation</h3>

<p><strong>You must complete this exercise in order to unlock the final project in this module. Your score on this assignment will not be included in the final grade calculation.</strong><p>

<p>You are expected to write code where you see <em># YOUR CODE HERE</em> within the cells of this notebook. Not all cells will be graded; code input cells followed by cells marked with <em>#Autograder test cell</em> will be graded. Upon submitting your work, the code you write at these designated positions will be assessed using an "autograder" that will run all test cells to assess your code. You will receive feedback from the autograder that will identify any errors in your code. Use this feedback to improve your code if you need to resubmit. Be sure not to change the names of any provided functions, classes, or variables within the existing code cells, as this will interfere with the autograder. Also, remember to execute all code cells sequentially, not just those you’ve edited, to ensure your code runs properly.</p>
    
<p>You can resubmit your work as many times as necessary before the submission deadline. If you experience difficulty or have questions about this exercise, use the Q&A discussion board to engage with your peers or seek assistance from the instructor.<p>

<p>Before starting your work, please review <a href="https://s3.amazonaws.com/ecornell/global/eCornellPlagiarismPolicy.pdf">eCornell's policy regarding plagiarism</a> (the presentation of someone else's work as your own without source credit).</p>

<h3>Submit Code for Autograder Feedback</h3>

<p>Once you have completed your work on this notebook, you will submit your code for autograder review. Follow these steps:</p>

<ol>
  <li><strong>Save your notebook.</strong></li>
  <li><strong>Mark as Completed —</strong> In the blue menu bar along the top of this code exercise window, you’ll see a menu item called <strong>Education</strong>. In the <strong>Education</strong> menu, click <strong>Mark as Completed</strong> to submit your code for autograder/instructor review. This process will take a moment and a progress bar will show you the status of your submission.</li>
	<li><strong>Review your results —</strong> Once your work is marked as complete, the results of the autograder will automatically be presented in a new tab within the code exercise window. You can click on the assessment name in this feedback window to see more details regarding specific feedback/errors in your code submission.</li>
  <li><strong>Repeat, if necessary —</strong> The Jupyter notebook will always remain accessible in the first tabbed window of the exercise. To reattempt the work, you will first need to click <strong>Mark as Uncompleted</strong> in the <strong>Education</strong> menu and then proceed to make edits to the notebook. Once you are ready to resubmit, follow steps one through three. You can repeat this procedure as many times as necessary.</li>
</ol>

<h2>Import NumPy and Check Python Version</h2>

First, you must import NumPy. Let's also check our version of Python. We've added the code for you for this first step.

In [1]:
import sys
import numpy as np # Numpy is Python's built in library for matrix operations.
from pylab import * 
sys.path.append('/home/codio/workspace/.guides/hf')
from helper import *
print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.6.7



<h2> Euclidean Distances in Python </h2>

<p>Many machine learning algorithms access their input data primarily through pairwise (Euclidean) distances, therefore it is important that we have a fast function that computes pairwise distances of input vectors.</p>
<p>Assume we have $n$ data vectors $\mathbf{x_1},\dots,\mathbf{x_n}\in{\cal R}^d$ and $m$ vectors $\mathbf{z_1},\dots,z_m\in{\cal R}^d$. With these data vectors, let us define two matrices $X=[\mathbf{x_1},\dots,\mathbf{x_n}]\in{\cal R}^{n\times d}$, where the $i^{th}$ row is a vector $\mathbf{x_i}$ and similarly $Z=[\mathbf{z_1},\dots,\mathbf{z_m}]\in{\cal R}^{m\times d}$. </p>
<p>We want a distance function that takes as input these two matrices $X$ and $Z$ and outputs a matrix $D\in{\cal R}^{n\times m}$, where 
	$$D_{ij}=\sqrt{(\mathbf{x_i}-\mathbf{z_j})(\mathbf{x_i}-\mathbf{z_j})^\top}.$$
</p>

A naïve implementation to compute pairwise distances may look like the code below:

In [2]:
def l2distanceSlow(X,Z=None):
    if Z is None:
        Z = X
    
    n, d = X.shape     # dimension of X
    m= Z.shape[0]   # dimension of Z
    D=np.zeros((n,m)) # allocate memory for the output matrix
    for i in range(n):     # loop over vectors in X
        for j in range(m): # loop over vectors in Z
            D[i,j]=0.0; 
            for k in range(d): # loop over dimensions
                D[i,j]=D[i,j]+(X[i,k]-Z[j,k])**2; # compute l2-distance between the ith and jth vector
            D[i,j]=np.sqrt(D[i,j]); # take square root
    return D

Please read through the code above carefully and make sure you understand it. It is perfectly correct and will produce the correct result... eventually. To see what is wrong, try running the <code>l2distanceSlow</code> code below on an extremely small matrix <code>X</code>.


In [3]:
X=np.random.rand(700,100)
print("Running the naive version for the...")
%time Dslow=l2distanceSlow(X)

Running the naive version for the...
CPU times: user 39.8 s, sys: 158 ms, total: 40 s
Wall time: 40 s


This code defines some random data in $X$ and computes the corresponding distance matrix $D$. The <em>%time</em> statement determines how long this code takes to run. This implementation is much too slow for such a simple operation on a small amount of data, and writing code like this to deal with matrices in this course will result in code that takes <strong>days</strong> to run.

<strong>As a general rule, you should avoid tight loops at all cost.</strong> As you will see in the remainder of this exercise, you can do much better by performing bulk matrix operations using the NumPy package, which calls highly optimized compiled code behind the scenes.





<h2> Efficient Programming with NumPy </h2>

<p>Although there is an execution overhead per line in Python, matrix operations are optimized and fast. In order to successfully program in this course, you need to free yourself from "for-loop" thinking and start thinking in terms of matrix operations. Python for scientific computing can be very fast if almost all the time is spent on a few heavy duty matrix operations. In this exercise, you will transform the function above into a few matrix operations <em>without any loops at all.</em> </p> 

<p>The key to efficient programming in Python for machine learning in general is to think about it in terms of mathematics and not in terms of loops. </p>

<h2>Exercises</h2>

<p>In the following three exercises, you'll take the steps necessary to implement the euclidean distance function without loops.</p>

<h3>Exercise 1: Inner-Product Matrix</h3>

<p>Show that the Inner-Product Matrix (Gram matrix) can be expressed in terms of pure matrix multiplication.

$$	G_{ij}=\mathbf{x}_i\mathbf{z}_j^\top $$

Once you are done with the derivation, implement the function <strong><code>innerproduct</code></strong>.</p>

In [4]:
def innerproduct(X,Z=None):
    # function innerproduct(X,Z)
    #
    # Computes the inner-product matrix.
    # Syntax:
    # D=innerproduct(X,Z)
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix G of size nxm
    # G[i,j] is the inner-product between vectors X[i,:] and Z[j,:]
    #
    # call with only one input:
    # innerproduct(X)=innerproduct(X,X)
    #
    if Z is None: # case when there is only one input (X)
        Z=X;

    # YOUR CODE HERE
    G = X.dot(Z.T)
    return G
#     raise NotImplementedError()

In [5]:
#Run this self-test cell to check your code

def innerprod_0():
    # test the output dimensions of innerproduct with one input matrix
    X = np.random.rand(700,10) # define 700 random inputs X
    test = (innerproduct(X).shape==700,700)    # check if inner-product matrix has dimension 700x700
    return test

def innerprod_1():
    # test the output dimensions of innerproduct with two matrices
    X = np.random.rand(700,10) # define 700 random inputs X
    Z = np.random.rand(200,10) # define 200 random inputs Z 
    test=(innerproduct(X,Z).shape ==(700,200)) # check if inner-product matrix has dimensions 700x200
    return test

def innerprod_2():
    X = np.random.rand(700,100) # define 700 random inputs X
    IP1 = innerproduct(X) # compute inner-product matrix with YOUR code
    IP2 = innerproduct_grader(X) # compute inner-product matrix with OUR code
    test = np.linalg.norm(IP1 - IP2) # compute the norm of the difference
    return test<1e-5 # this norm should be essentially 0

def innerprod_3():
    X = np.random.rand(700,100) # define 700 random inputs X
    Z = np.random.rand(300,100) # define 300 random inputs X
    IP1 = innerproduct(X,Z) # compute inner-product matrix with YOUR code
    IP2 = innerproduct_grader(X,Z) # compute inner-product matrix with OUR code
    test = np.linalg.norm(IP1 - IP2) # compute the norm of the difference
    return test<1e-5 # this norm should be essentially 0


runtest(innerprod_0,'innerprod_0')
runtest(innerprod_1,'innerprod_1')
runtest(innerprod_2,'innerprod_2')
runtest(innerprod_3,'innerprod_3')

Running Test: innerprod_0 ... ✔ Passed!
Running Test: innerprod_1 ... ✔ Passed!
Running Test: innerprod_2 ... ✔ Passed!
Running Test: innerprod_3 ... ✔ Passed!


In [6]:
# Autograder test cell - worth 1 Point
# runs innerprod0

In [7]:
# Autograder test cell - worth 1 Point
# runs innerprod1

In [8]:
# Autograder test cell - worth 1 Point
# runs innerprod2

In [9]:
# Autograder test cell - worth 1 Point
# runs innerprod3

<h3>Exercise 2: Derive the Distance Matrix</h3>

Let us define two new matrices $S,R\in{\cal R}^{n\times m}$ 
		$$S_{ij}=\mathbf{x}_i\mathbf{x}_i^\top, \ \ R_{ij}=\mathbf{z}_j\mathbf{z}_j^\top.$$
 	Show that the <em>squared</em>-euclidean matrix $D^2\in{\cal R}^{n\times m}$, defined as
		$$D^2_{ij}=(\mathbf{x}_i-\mathbf{z}_j)(\mathbf{x}_i-\mathbf{z}_j)^\top,$$
	can be expressed as a linear combination of the matrix $S, G, R$. (Hint: It might help to first express $D^2_{ij}$ in terms of inner-products.) What do you need to do to obtain the true Euclidean distance matrix $D$?</p></td>
	
<p>

<h3>Exercise 3: Implement <code>l2distance</code></h3>

<p>Implement the function <strong><code>l2distance</code></strong>, which computes the Euclidean distance matrix $D$ without a single loop. (Hint: Make sure that when you take the square root of the squared distance matrix, ensure that all entries are non-negative. Sometimes very small numvers can be non-negative due to numerical precision, you can just set them to 0.)</p>

In [10]:
def l2distance(X,Z=None):
    # function D=l2distance(X,Z)
    #
    # Computes the Euclidean distance matrix.
    # Syntax:
    # D=l2distance(X,Z)
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix D of size nxm
    # D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    #
    # call with only one input:
    # l2distance(X)=l2distance(X,X)
    #
    if Z is None:
        Z=X;

    n,d1=X.shape
    m,d2=Z.shape
    assert (d1==d2), "Dimensions of input vectors must match!"

    # YOUR CODE HERE
    G = X.dot(Z.T)
    S =np.sum(Z**2, axis=1).reshape(m,1)
    R = np.sum(X**2, axis=1).reshape(n,1)
    Dsq = R+S.T-2*G
    D = np.sqrt(Dsq.clip(0))
    
    return D
#     raise NotImplementedError()

In [11]:
#Run this self-test cell to check your code

def distance_accuracy(): 
    X = np.random.rand(700,100) # define random inputs
    D1 = l2distance(X) # compute distances from your solutions
    D2 = l2distance_grader(X) #compute distance from ground truth
    test = np.linalg.norm(D1 - D2) # compare the two
    return test<1e-5 # difference should be small

def distance_squareroot():  
    X = np.random.rand(700,100) # define random inputs
    D1 = l2distance(X) # compute distances from your solutions
    D2sq = l2distance_grader(X)**2 #compute distance from ground truth *but square them*
    test = np.linalg.norm(D1 - D2sq) # compare the two
    return test>1e-5 # difference should be big

def dimensions():
    X = np.random.rand(700,100) # define random inputs
    Z = np.random.rand(800,100) # define random inputs
    n,d1=X.shape
    m,d2=Z.shape    
    D1 = l2distance(X,Z) # compute distances from your solutions
    o1,o2=D1.shape
    return (o1==n) and (o2==m)

def matrix_dist_accuracy():
    X = np.random.rand(700,100)
    Z = np.random.rand(300,100)
    D1Z = l2distance(X,Z)
    D2Z = l2distance_grader(X,Z)
    test = np.linalg.norm(D1Z - D2Z)
    return test<1e-5

runtest(distance_accuracy,'distance_accuracy')
runtest(distance_squareroot,'distance_squareroot')
runtest(dimensions,'dimensions')
runtest(matrix_dist_accuracy,'matrix_dist_accuracy')

Running Test: distance_accuracy ... ✔ Passed!
Running Test: distance_squareroot ... ✔ Passed!
Running Test: dimensions ... ✔ Passed!
Running Test: matrix_dist_accuracy ... ✔ Passed!


In [12]:
# Autograder test cell - worth 1 Point
# runs distance_accuracy

In [13]:
# Autograder test cell - worth 1 Point
# runs distance_squareroot

In [14]:
# Autograder test cell - worth 1 Point
# runs dimensions

In [15]:
# Autograder test cell - worth 1 Point
# runs matrix_dist_accuracy

Let's now compare the speed of your l2-distance function against the previous naïve implementation:

In [16]:
import time
current_time = lambda: int(round(time.time() * 1000))

X=np.random.rand(700,100)
Z=np.random.rand(300,100)

print("Running the naïve version...")
before = current_time()
Dslow=l2distanceSlow(X)
after = current_time()
t_slow = after - before
print("{:2.0f} ms".format(t_slow))

print("Running the vectorized version...")
before = current_time()
Dfast=l2distance(X)
after = current_time()
t_fast = after - before
print("{:2.0f} ms".format(t_fast))


speedup = t_slow / t_fast
print("The two methods should deviate by very little: {:05.6f}".format(norm(Dfast-Dslow)))
print("but your NumPy code was {:05.2f} times faster!".format(speedup))

Running the naïve version...
42093 ms
Running the vectorized version...
16 ms
The two methods should deviate by very little: 0.000002
but your NumPy code was 2630.81 times faster!


How much faster is your code now? With this implementation, you should easily be able to compute the distances between <strong>many more</strong> vectors. It should be clear now, even for small datasets, that the for-loop based implementation could take several days or even weeks to perform basic operations that take seconds or minutes with well-written NumPy code.